# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [66]:
!cat publications.tsv

pub_date	title	venue	excerpt	authors	url_slug	paper_url
2017-01-01	Spatial Projection of Multiple Climate Variables Using Hierarchical Multitask Learning	AAAI'17	This paper proposes a new Hierarchical Multitask Learning framework for the joint prediction of multiple climate variables. Each task in this multitask learning set up is actually another multitask learning problem.	AR Goncalves, A Banerjee, FJ Von Zuben	paper-hmtl-aaai17	http://www.aaai.org/ocs/index.php/AAAI/AAAI17/paper/download/15018/14060
2016-01-01	Multi-task sparse structure learning with gaussian copula models	JMLR	This paper presents a novel family of models for MTL, applicable to regression and classification problems, capable of learning the structure of tasks relationship via Copula distributions.	AR Goncalves, FJ Von Zuben	paper-mssl-cop	http://www.jmlr.org/papers/volume17/15-215/15-215.pdf
2018-06-01	Modeling Alzheimer's disease cognitive scores using multi-task sparse group lasso	Computerized Medical Imaging and

## Import pandas

We are using the very handy pandas library for dataframes.

In [67]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [68]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,authors,url_slug,paper_url
0,2017-01-01,Spatial Projection of Multiple Climate Variabl...,AAAI'17,This paper proposes a new Hierarchical Multita...,"AR Goncalves, A Banerjee, FJ Von Zuben",paper-hmtl-aaai17,http://www.aaai.org/ocs/index.php/AAAI/AAAI17/...
1,2016-01-01,Multi-task sparse structure learning with gaus...,JMLR,This paper presents a novel family of models f...,"AR Goncalves, FJ Von Zuben",paper-mssl-cop,http://www.jmlr.org/papers/volume17/15-215/15-...
2,2018-06-01,Modeling Alzheimer's disease cognitive scores ...,Computerized Medical Imaging and Graphics (Jou...,We consider the problem of simultaneously pred...,"X Liu, AR Goncalves, P Cao, D Zhao, A Banerjee",paper-group-lasso,https://www.sciencedirect.com/science/article/...
3,2017-09-01,On the generalization of fused systems in voi...,16th International Conference of the Biometric...,This paper describes presentation attack detec...,"AR Goncalves, P Korshunov, RPV Violato, FO Sim...",paper-asvspoof,https://ieeexplore.ieee.org/document/8053516/
4,2017-08-01,Structured Estimation in High Dimensions and ...,Large-Scale Machine Learning in the Earth Scie...,We discuss structured sparse regression and m...,"S Chatterjee, V Sivakumar, AR Goncalves, A Ban...",paper-book-chapter,https://www.taylorfrancis.com/books/e/97814987...
5,2015-06-27,Multi-Label Structure Learning with Ising Mode...,IJCAI'15,Built on recent advances in structure learnin...,"AR Goncalves, FJ Von Zuben, A Banerjee",paper-ijcai-2015,http://www.aaai.org/ocs/index.php/IJCAI/IJCAI1...
6,2014-11-3,Multi-task Sparse Structure Learning,CIKM'14,"In this paper, we present a novel family of mo...","AR Goncalves, P Das, S Chatterjee, V Sivakumar...",paper-cikm-2014,https://dl.acm.org/citation.cfm?id=2662091


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [69]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [82]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    md += "\nauthors: '" + html_escape(item.authors) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page

    md += "\nAuthors: " + html_escape(item.authors) + "\n"

    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.paper_url)) > 5:
        md += "\n[Access full paper here](" + item.paper_url + ")\n" 
        
#     md += "\nRecommended citation: " + item.citation
    print(md)
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

---
title: "Spatial Projection of Multiple Climate Variables Using Hierarchical Multitask Learning"
collection: publications
permalink: /publication/2017-01-01-paper-hmtl-aaai17
excerpt: 'This paper proposes a new Hierarchical Multitask Learning framework for the joint prediction of multiple climate variables. Each task in this multitask learning set up is actually another multitask learning problem.'
date: 2017-01-01
venue: 'AAAI&apos;17'
authors: 'AR Goncalves, A Banerjee, FJ Von Zuben'
paperurl: 'http://www.aaai.org/ocs/index.php/AAAI/AAAI17/paper/download/15018/14060'
---
Authors: AR Goncalves, A Banerjee, FJ Von Zuben

This paper proposes a new Hierarchical Multitask Learning framework for the joint prediction of multiple climate variables. Each task in this multitask learning set up is actually another multitask learning problem.

[Access full paper here](http://www.aaai.org/ocs/index.php/AAAI/AAAI17/paper/download/15018/14060)

---
title: "Multi-task sparse structure learning wi

These files are in the publications directory, one directory below where we're working from.

In [75]:
!ls ../_publications/

2014-11-3-paper-cikm-2014.md     2017-08-01-paper-book-chapter.md
2015-06-27- paper-ijcai-2015.md  2017-09-01-paper-asvspoof.md
2016-01-01-paper-mssl-cop.md     2018-06-01-paper-group-lasso.md
2017-01-01-paper-hmtl-aaai17.md


In [72]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
